# Using EcoFOCIpy to process raw field data

## DY2307 (Spring Mooring Cruise - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [1]:
import yaml
import glob

import ecofocipy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import ecofocipy.io.ncCFsave as ncCFsave
import ecofocipy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [2]:
sample_data_dir = '/Users/bell/ecoraid/2023/CTDcasts/dy2307/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [3]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY2307' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY2307.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2023/CTDcasts/dy2307/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2307/rawconverted/ctd002.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2307/rawconverted/ctd003.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2307/rawconverted/ctd004.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2307/rawconverted/ctd005.btl


In [4]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd001.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sal00,sal11,sigma-t00,sigma-t11,sbeox1ml/l,sbeox1ps,sbox1mm/kg,fleco-afl,t090c,t190c,prdm,turbwetntu0,par,scan,datetime
count,8.000000,8.00000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8
mean,7.609475,103.73975,331.469625,31.860650,31.864287,25.254587,25.257262,7.662987,104.468750,333.800875,0.407100,4.349287,4.351100,42.787625,0.193912,278.369642,14538.875000,2023-05-17 15:45:32
min,7.074400,95.82600,308.103000,31.752000,31.755100,25.091100,25.091900,7.119200,96.433000,310.053000,0.114100,4.007800,4.007800,2.940000,0.122500,0.020437,7723.000000,2023-05-17 15:40:48
25%,7.351350,99.42975,320.192750,31.757075,31.761000,25.172100,25.175075,7.406700,100.177750,322.602750,0.182900,4.033050,4.033025,18.584750,0.156950,3.754375,11172.250000,2023-05-17 15:43:11.500000
50%,7.574300,103.45600,329.953000,31.802450,31.806100,25.198200,25.201100,7.621500,104.100500,332.010500,0.275900,4.365600,4.365400,36.334500,0.205150,21.432500,14700.000000,2023-05-17 15:45:39
75%,7.948825,108.33925,346.277750,31.946150,31.949850,25.356500,25.359400,8.013800,109.225250,349.111000,0.578950,4.455400,4.455325,57.461750,0.219675,194.542500,17693.500000,2023-05-17 15:47:43.500000
max,8.154700,113.06200,355.276000,32.062600,32.066200,25.446200,25.449100,8.203500,113.738000,357.400000,1.005700,5.091700,5.106800,106.770000,0.259000,1645.100000,21171.000000,2023-05-17 15:50:08
std,0.401512,6.13821,17.529036,0.128199,0.128220,0.131028,0.131376,0.407090,6.208112,17.772946,0.321743,0.358909,0.363385,34.771230,0.047310,569.665581,4706.673179,NaN


## Create BTL report file

In [5]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [6]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY2307',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'NOAAS Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2023, 5, 14),
 'EndDate': datetime.date(2023, 5, 24),
 'Project': 'FOCI',
 'ChiefScientist': 'Jesse Lamb',
 'StartPort': 'Dutch Harbor, AK',
 'EndPort': 'Kodiak, AK',
 'CruiseLocation': 'Gulf of Alaska',
 'Description': 'FOCI Spring Icthy Survey',
 'CruiseYear': 2023,
 'ctdlogs_pdf_name': ''}

In [7]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 54237,
 'Vessel': 'NOAAS Oscar Dyson',
 'CruiseID': 'DY2307',
 'Project_Leg': '',
 'UniqueCruiseID': 'DY2307',
 'Project': 'EcoFOCI Spring Icthy Cruise',
 'StationNo_altname': '',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 55,
 'LatitudeMin': 40.09,
 'LongitudeDeg': 158,
 'LongitudeMin': 34.91,
 'GMTDay': 17,
 'GMTMonth': 'May',
 'GMTYear': 2023,
 'GMTTime': 84900,
 'DryBulb': -99.9,
 'RelativeHumidity': -999,
 'WetBulb': -99.9,
 'Pressure': -9999,
 'SeaState': '',
 'Visibility': '',
 'WindDir': -999,
 'WindSpd': -99.0,
 'CloudAmt': '',
 'CloudType': '',
 'Weather': '',
 'SurfaceTemp': -99.9,
 'BottomDepth': -9999,
 'StationNameID': '',
 'MaxDepth': -9999,
 'InstrumentSerialNos': '',
 'Notes': '',
 'NutrientBtlNiskinNo': '',
 'NutrientBtlNumbers': '',
 'OxygenBtlNiskinNo': '',
 'OxygenBtlNumbers': '',
 'SalinityBtlNiskinNo': '',
 'SalinityBtlNumbers': '',
 'ChlorophyllBtlNiskinNo': '',
 'ChlorophyllBtlVolumes': '',
 'InstrumentType': 'SBE911plus',
 'WaterMassCode': 'B',
 'i

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [8]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [9]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd001.btl'] = cruise_data['ctd001.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag'})

cruise_data['ctd001.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,oxy_concM_ch2,oxy_percentsat_ch2,oxy_conc_ch2,chlor_fluorescence,temperature_ch1,temperature_ch2,Pressure [dbar],turbidity,par,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,,
7.0,7.9234,108.031,345.171,31.7575,31.7615,25.1688,25.172,7.986,108.885,347.901,0.2652,4.3942,4.3938,11.504,0.1982,411.21,19603.0,2023-05-17 15:49:03,ctd001


In [10]:
cruise_data['ctd001.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'salinity_ch1',
       'salinity_ch2', 'sigma_t_ch1', 'sigma_t_ch2', 'oxy_concM_ch2',
       'oxy_percentsat_ch2', 'oxy_conc_ch2', 'chlor_fluorescence',
       'temperature_ch1', 'temperature_ch2', 'Pressure [dbar]', 'turbidity',
       'par', 'scan', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [11]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [12]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [13]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0
Data variables: (12/19)
    oxy_concM_ch1       (bottle) float64 7.074 7.122 7.428 ... 8.025 7.923 8.155
    oxy_percentsat_ch1  (bottle) float64 95.83 96.38 100.4 ... 109.3 108.0 113.1
    oxy_conc_ch1        (bottle) float64 308.1 310.2 323.5 ... 349.6 345.2 355.3
    salinity_ch1        (bottle) float64 32.06 32.04 31.92 ... 31.76 31.76 31.75
    salinity_ch2        (bottle) float64 32.07 32.04 31.92 ... 31.76 31.76 31.76
    sigma_t_ch1         (bottle) float64 25.45 25.43 25.33 ... 25.17 25.17 25.09
    ...                  ...
    Pressure [dbar]     (bottle) float64 106.8 76.29 51.19 ... 20.95 11.5 2.94
    turbidity           (bottle) float64 0.259 0.2127 0.1225 ... 0.1982 0.2406
    par                 (bottle) float64 0.02044 0.6538 ... 411.2 1.645e+03
    scan                (bottle) float64 7.723e+03 9.691e+03 ... 2.117e+04
    datetime            (bottle) datetime64[ns] 2023-05-17T15:40:48 ... 2023-...
    cast                (bottle) object 'ctd001' 'ctd001' ... 'ctd001' 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [14]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [15]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd001.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [16]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/19)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 7.074 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 95.83 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 308.1 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 32.06 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 32.07 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.45 ......
    ...                  ...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 106.8 ......
    turbidity           (latitude, longitude, time, bottle) float64 0.259 ......
    par                 (latitude, longitude, time, bottle) float64 0.02044 ....
    scan                (latitude, longitude, time, bottle) float64 7.723e+03...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....

In [17]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/19)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 7.074 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 95.83 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 308.1 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 32.06 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 32.07 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.45 ......
    ...                  ...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 106.8 ......
    turbidity           (latitude, longitude, time, bottle) float64 0.259 ......
    par                 (latitude, longitude, time, bottle) float64 0.02044 ....
    scan                (latitude, longitude, time, bottle) float64 7.723e+03...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....
Attributes: (12/44)
    CruiseID:                      DY2307
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        NOAAS Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2023-07-07T15:16:07Z
    date_modified:                 
    QC_indicator:                  unqcd

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [18]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history